In [1]:
import breeze.linalg._
import breeze.stats.distributions._
import io.github.mandar2812.dynaml.pipes._
import io.github.mandar2812.dynaml.kernels._
import io.github.mandar2812.dynaml.models.bayes.{
    LinearTrendESGPrior,
    LinearTrendGaussianPrior,
    GaussianProcessPrior
}
import io.github.mandar2812.dynaml.modelpipe._
import io.github.mandar2812.dynaml.probability._
import io.github.mandar2812.dynaml.DynaMLPipe._
import io.github.mandar2812.dynaml.graphics.charts.Highcharts._
import io.github.mandar2812.dynaml.analysis.implicits._
import io.github.mandar2812.dynaml.optimization.GPMixtureMachine
import io.github.mandar2812.dynaml.pipes.Encoder
import io.github.mandar2812.dynaml.probability.distributions.UnivariateGaussian
import io.github.mandar2812.dynaml.analysis._
import io.github.mandar2812.dynaml.analysis.implicits._
import spire.implicits._
import io.github.mandar2812.dynaml.graphics.charts.Highcharts._
import io.github.mandar2812.dynaml.tensorflow.dtfdata

import breeze.linalg._

import breeze.stats.distributions._

import io.github.mandar2812.dynaml.pipes._

import io.github.mandar2812.dynaml.kernels._

import io.github.mandar2812.dynaml.models.bayes.{
    LinearTrendESGPrior,
    LinearTrendGaussianPrior,
    GaussianProcessPrior
}

import io.github.mandar2812.dynaml.modelpipe._

import io.github.mandar2812.dynaml.probability._

import io.github.mandar2812.dynaml.DynaMLPipe._

import io.github.mandar2812.dynaml.graphics.charts.Highcharts._

import io.github.mandar2812.dynaml.analysis.implicits._

import io.github.mandar2812.dynaml.optimization.GPMixtureMachine

import io.github.mandar2812.dynaml.pipes.Encoder

import io.github.mandar2812.dynaml.probability.distributions.UnivariateGaussian

import io.github.mandar2812.dynaml.analysis._

import io.github.mandar2812.dynaml.analysis.implicits._

import spire.implicits._

import io.github.mandar2812.dynaml.graphics.charts.Highcharts._

import io.github.mandar2812.dynaml.tensorflow.dtfdata

In [2]:
  //For p order auto-regressive dynamics
  val p                = 3
  val num_sample_paths = 10
  //Simulation time of the time series models.
  val len = 100

  val xs = Seq.tabulate[Double](len)(1d * _)

  implicit val ev = VectorField(p * (p + 1))

  val y0 = RandomVariable(UnivariateGaussian(0.0, 1.0))

  val urv = UniformRV(-1.0, 1.0)

  val coeff_gen =
    (order: Int) =>
      MultGaussianRV(
        DenseVector.tabulate[Double](order * (order + 1))(
          i => if (i == 0) 0d else -0.01d
        ),
        diag(
          DenseVector.tabulate[Double](order * (order + 1))(
            i =>
              if (i < order) 0.01d
              else 0.001
          )
        )
      )

  val w_prior = coeff_gen(p)

  //Draw the AR(p) coefficients from the prior distribution w_prior
  val w = w_prior.draw


Apr 28, 2020 3:19:37 PM com.github.fommil.jni.JniLoader liberalLoad
INFO: successfully loaded /tmp/jniloader6702601265569287600netlib-native_system-linux-x86_64.so
Apr 28, 2020 3:19:37 PM com.github.fommil.jni.JniLoader load
INFO: already loaded netlib-native_system-linux-x86_64.so


p: Int = 3
num_sample_paths: Int = 10
len: Int = 100
xs: Seq[Double] = List(
  0.0,
  1.0,
  2.0,
  3.0,
  4.0,
  5.0,
  6.0,
  7.0,
  8.0,
  9.0,
  10.0,
  11.0,
  12.0,
  13.0,
  14.0,
  15.0,
  16.0,
  17.0,
  18.0,
  19.0,
  20.0,
  21.0,
  22.0,
  23.0,
  24.0,
  25.0,
  26.0,
  27.0,
  28.0,
  29.0,
  30.0,
  31.0,
  32.0,
  33.0,
  34.0,
  35.0,
  36.0,
  37.0,
...
ev: VectorField = io.github.mandar2812.dynaml.analysis.VectorField@61820694
y0: ContinuousRVWithDistr[Double, ContinuousDistr[Double]] = io.github.mandar2812.dynaml.probability.RandomVariable$$anon$19@50553f33
urv: UniformRV = UniformRV(-1.0, 1.0)
coeff_gen: Int => MultGaussianRV = ammonite.$sess.cmd1$Helper$$Lambda$2109/1623719933@6765ffb1
w_prior: MultGaussianRV = MultGaussianRV(
  DenseVector(0.0, -0.01, -0.01, -0.01, -0.01, -0.01, -0.01, -0.01, -0.01, -0.01, -0.01, -0.01),
  0.01  0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    
0.0   0.01  0.0   0.0    0.0    0.0    0.0    0.0    0.0   

In [3]:
  //Define some kernels for use later.
  val rbfc      = new SEKernel(1d, 2.0)
  val mlpKernel = new MLPKernel(1d, 0.5d)
  val fbm       = new FBMCovFunction(0.5)
  val stKernel  = new TStudentKernel(0.5)

  val perKernel = new PeriodicCovFunc(2d, 1.5d, 0.1d)
  val arpKernel = new GenericMaternKernel[Double](2.5, p)
  val n         = new MAKernel(1.0)
  val noise     = new DiracKernel(0.5d)

  val gsmKernel = GaussianSpectralKernel(
    DenseVector.zeros[Double](p),
    DenseVector.ones[Double](p),
    GaussianSpectralKernel.getEncoderforBreezeDV(p)
  )

rbfc: SEKernel = io.github.mandar2812.dynaml.kernels.SEKernel@3d9b6fac
mlpKernel: MLPKernel = io.github.mandar2812.dynaml.kernels.MLPKernel@e79c6f0
fbm: FBMCovFunction = io.github.mandar2812.dynaml.kernels.FBMCovFunction@12f19590
stKernel: TStudentKernel = io.github.mandar2812.dynaml.kernels.TStudentKernel@2e739e54
perKernel: PeriodicCovFunc = io.github.mandar2812.dynaml.kernels.PeriodicCovFunc@34b96543
arpKernel: GenericMaternKernel[Double] = io.github.mandar2812.dynaml.kernels.GenericMaternKernel@535f111d
n: MAKernel = io.github.mandar2812.dynaml.kernels.MAKernel@5dddc50a
noise: DiracKernel = io.github.mandar2812.dynaml.kernels.DiracKernel@2a6ba5b4
gsmKernel: GaussianSpectralKernel[DenseVector[Double]] = io.github.mandar2812.dynaml.kernels.GaussianSpectralKernel@19616a98

In [4]:
  val linear_coeff    = (0d, 0d)
  val quadratic_coeff = (-0.01d, -0.75d, 0d)

  //Define the trend functions. One a linear trend
  //and the other a parabola.

  val linear_vec_trend = MetaPipe(
    (p: DenseVector[Double]) =>
      (x: DenseVector[Double]) => {
        p dot x
      }
  )

  val basis_func_mapping = DataPipe(
    (x: DenseVector[Double]) =>
      (x * DenseVector.vertcat(DenseVector(1d), x).t).toDenseVector
  )

  val quadratic_vec_trend = MetaPipe(
    (p: DenseVector[Double]) =>
      (x: DenseVector[Double]) => {
        p dot basis_func_mapping(x)
      }
  )

  val linear_trend_mean = MetaPipe(
    (p: (Double, Double)) =>
      (x: Double) => {
        (p._1 * x) + p._2
      }
  )

  val quadratic_trend_mean = MetaPipe(
    (p: (Double, Double, Double)) =>
      (x: Double) => {
        (p._1 * x * x) + (p._2 * x) + p._3
      }
  )

  //Determine how the trend coefficients map to key-value pairs
  val linear_trend_encoder = Encoder(
    (cs: (Double, Double)) => Map("slope" -> cs._1, "intercept" -> cs._2),
    (conf: Map[String, Double]) => (conf("slope"), conf("intercept"))
  )

  val linear_vec_trend_encoder = Encoder(
    (cs: DenseVector[Double]) => {
      cs.toArray.zipWithIndex.map(cp => (s"w_${cp._2}", cp._1)).toMap
    },
    (conf: Map[String, Double]) =>
      DenseVector(conf.toSeq.sortBy(_._1).map(_._2).toArray)
  )

  val quadratic_trend_encoder = Encoder(
    (cs: (Double, Double, Double)) =>
      Map("a" -> cs._1, "b" -> cs._2, "d" -> cs._3),
    (conf: Map[String, Double]) => (conf("a"), conf("b"), conf("d"))
  )


linear_coeff: (Double, Double) = (0.0, 0.0)
quadratic_coeff: (Double, Double, Double) = (-0.01, -0.75, 0.0)
linear_vec_trend: MetaPipe[DenseVector[Double], DenseVector[Double], Double] = io.github.mandar2812.dynaml.pipes.MetaPipe$$anon$2@1ee7bc89
basis_func_mapping: DataPipe[DenseVector[Double], DenseVector[Double]] = io.github.mandar2812.dynaml.pipes.DataPipe$$anon$2@160a649c
quadratic_vec_trend: MetaPipe[DenseVector[Double], DenseVector[Double], Double] = io.github.mandar2812.dynaml.pipes.MetaPipe$$anon$2@6e8a25cc
linear_trend_mean: MetaPipe[(Double, Double), Double, Double] = io.github.mandar2812.dynaml.pipes.MetaPipe$$anon$2@63da6fa6
quadratic_trend_mean: MetaPipe[(Double, Double, Double), Double, Double] = io.github.mandar2812.dynaml.pipes.MetaPipe$$anon$2@3bcf29e7
linear_trend_encoder: Encoder[(Double, Double), Map[String, Double]] = io.github.mandar2812.dynaml.pipes.Encoder$$anon$1@20fd73b2
linear_vec_trend_encoder: Encoder[DenseVector[Double], Map[String, Double]] = io.github.m

In [5]:
  //Define a Matern(p + 1/2) covariance based gaussian process
  val gp_explicit = GaussianProcessPrior[Double, (Double, Double)](
    arpKernel,
    new MAKernel(0.5d),
    linear_trend_mean,
    linear_trend_encoder,
    linear_coeff
  )

  //Define a gaussian process for GP Time Series models.
  val gp_prior = GaussianProcessPrior[DenseVector[Double], DenseVector[Double]](
    rbfc,
    noise,
    linear_vec_trend,
    linear_vec_trend_encoder,
    w(0 until p)
  )

  val gpModelPipe =
    new GPBasisFuncRegressionPipe[Seq[(DenseVector[Double], Double)], DenseVector[
      Double
    ]](
      identityPipe[Seq[(DenseVector[Double], Double)]],
      rbfc,
      noise,
      identityPipe[DenseVector[Double]],
      w_prior(0 until p)
    )

gp_explicit: GaussianProcessPrior[Double, (Double, Double)] = io.github.mandar2812.dynaml.models.bayes.GaussianProcessPrior$$anon$1@c0bb52e
gp_prior: GaussianProcessPrior[DenseVector[Double], DenseVector[Double]] = io.github.mandar2812.dynaml.models.bayes.GaussianProcessPrior$$anon$1@189f373b
gpModelPipe: GPBasisFuncRegressionPipe[Seq[(DenseVector[Double], Double)], DenseVector[Double]] = io.github.mandar2812.dynaml.modelpipe.GPBasisFuncRegressionPipe@c12cd89

In [6]:
  val y_explicit: MultGaussianPRV = gp_explicit.priorDistribution(xs)

  //Generate samples for GP process on explicit time. Matern(p + 1/2)
  val samples_gp_explicit =
    y_explicit.iid(10).draw.map(s => s.toBreezeVector.toArray.toSeq).toSeq

  //Generate samples for GP-NAR process in a recursive manner
  val ys_ar_rec = RandomVariable[Seq[Double]](() => {
    // Generate y0 and y1
    val u0: DenseVector[Double] = DenseVector.tabulate(p)(_ => y0.draw)
    val u1: Double              = gp_prior.priorDistribution(Seq(u0)).draw.toStream.head
    val d                       = Seq((u0, u1))
    val (_, xsamples) =
      (1 to xs.length - p - 1)
        .scanLeft((d, DenseVector(Array(u1) ++ u0(0 to -2).toArray)))(
          (dc, _) => {

            val (data, x) = dc
            val gpModel   = gpModelPipe(data)
            val y         = gpModel.predictiveDistribution(Seq(x)).draw.toStream.head
            (data :+ (x, y), DenseVector(Array(y) ++ x(0 to -2).toArray))
          }
        )
        .unzip

    u0.toArray.toSeq ++ xsamples.map(_(0))
  })

  //Generate samples from a conventional AR(p) process
  //with coefficients given by w
  val markov_process = (n: Int) =>
    RandomVariable[Seq[Double]](() => {
      val u0 = DenseVector.tabulate(p)(_ => y0.draw)
      val xs_tail = (1 to n - p - 1)
        .scanLeft(
          u0
        )((y: DenseVector[Double], _) => {
          val y_new: Double = quadratic_vec_trend(w)(y) + y0.draw
          DenseVector(Array(y_new) ++ y(0 to -2).toArray)
        })
        .toSeq
        .map(_(0))

      u0.toArray.toSeq ++ xs_tail
    })

  val samples_ar_rec = ys_ar_rec.iid(10).draw.toSeq

  val samples_markov = markov_process(xs.length).iid(10).draw.toSeq


y_explicit: MultGaussianPRV = MultGaussianPRV(
  io.github.mandar2812.dynaml.algebra.PartitionedVector@4bc0f48c,
  io.github.mandar2812.dynaml.algebra.PartitionedPSDMatrix@7c5c70fe
)
samples_gp_explicit: collection.immutable.Seq[Seq[Double]] = Stream(
  WrappedArray(
    0.1629444962719716,
    0.8241632177493847,
    -0.47015250748219467,
    0.7467483514359702,
    1.1374129903633636,
    0.9786814092389757,
    0.0657343046793386,
    -1.6207913298155512,
    -1.2795169592901643,
    -2.469232063180355,
    -0.6403631504769587,
    -1.0383734333106216,
    -0.32622104805137986,
    -1.2073837234085956,
    -0.27412266956447773,
    -0.2566966954773496,
    -0.16327764900854816,
    -1.9235383238618664,
    -0.7309674942239188,
    0.7764204472892756,
    1.445819442664622,
    1.7305920226878018,
    3.387817607036443,
    1.746628718714702,
    0.44205878317120817,
    -1.1943739368366753,
    0.04007530327206499,
    -0.014463434293722377,
    -2.2204757370160793,
    -1.899200652

In [7]:
deleteAll()
val plot_legend_labels =
    (1 to num_sample_paths).map(i => s"Path $i").toSeq

spline(xs, samples_gp_explicit.head)
hold()
samples_gp_explicit.tail.foreach((s: Seq[Double]) => spline(xs, s))
unhold()
title(
    s"""GP Explicit Time AR($p): ${gp_explicit.covariance.toString
      .split("\\.")
      .last}"""
)
legend(plot_legend_labels)

serving resources from: file:/home/mandar/Development/examples/notebooks/index-1588080017394.html


SLF4J: Class path contains multiple SLF4J providers.
SLF4J: Found provider [org.slf4j.log4j12.Log4j12ServiceProvider@6b0abeff]
SLF4J: Found provider [org.slf4j.helpers.NOPServiceProvider@59e4c690]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual provider is of type [org.slf4j.log4j12.Log4j12ServiceProvider@6b0abeff]


Server started: http://tail-box:42715/index-1588080017394.html
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).


plot_legend_labels: collection.immutable.Seq[String] = Vector(
  "Path 1",
  "Path 2",
  "Path 3",
  "Path 4",
  "Path 5",
  "Path 6",
  "Path 7",
  "Path 8",
  "Path 9",
  "Path 10"
)
res6_2: io.github.mandar2812.dynaml.graphics.charts.highcharts.Highchart = Highchart(
  List(
    Series(
      List(
        DataPair(0.0, 0.1629444962719716, None, None),
        DataPair(1.0, 0.8241632177493847, None, None),
        DataPair(2.0, -0.47015250748219467, None, None),
        DataPair(3.0, 0.7467483514359702, None, None),
        DataPair(4.0, 1.1374129903633636, None, None),
        DataPair(5.0, 0.9786814092389757, None, None),
        DataPair(6.0, 0.0657343046793386, None, None),
        DataPair(7.0, -1.6207913298155512, None, None),
        DataPair(8.0, -1.2795169592901643, None, None),
        DataPair(9.0, -2.469232063180355, None, None),
        DataPair(10.0, -0.6403631504769587, None, None),
        DataPair(11.0, -1.0383734333106216, None, None),
        DataPair(12.0, -0.326

In [8]:
val tuple_encoder =
    TupleIntegerEncoder(List(p, p + 1))

val markov_formula = w.toArray.toSeq
    .zipWithIndex
    .map(
        cp => {
            val indices = tuple_encoder.i(cp._2)

            if (cp._2 < p) {
              if (cp._1 >= 0d && cp._2 != 0) f"+${cp._1}%3.2f*y(t-${cp._2 + 1})"
              else f"${cp._1}%3.2f*y(t-${cp._2 + 1})"
            } else if (cp._1 >= 0d)
            f"+${cp._1}%3.2f*y(t-${indices.head + 1})*y(t-${indices.last})"
            else f"${cp._1}%3.2f*y(t-${indices.head + 1})*y(t-${indices.last})"
        }
    )
    .reduceLeft(_ ++ _)

spline(xs, samples_markov.head)
hold()
samples_markov.tail.foreach((s: Seq[Double]) => spline(xs, s))
unhold()
title(
    s"""AR($p): y(t) = ${markov_formula} + noise"""
)
legend(plot_legend_labels)

spline(xs, samples_ar_rec.head)
hold()
samples_ar_rec.tail.foreach((s: Seq[Double]) => spline(xs, s))
unhold()
title(
    s"""GP-AR Recurrent: ${gpModelPipe.covariance.toString.split("\\.").last}"""
)
legend(plot_legend_labels)

println(s"Linear coefficients: ${w.toArray.toSeq}")


Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output

tuple_encoder: TupleIntegerEncoder = TupleIntegerEncoder(List(3, 4))
markov_formula: String = "0.15*y(t-1)+0.07*y(t-2)-0.04*y(t-3)-0.02*y(t-1)*y(t-1)-0.04*y(t-2)*y(t-1)-0.00*y(t-3)*y(t-1)-0.00*y(t-1)*y(t-2)+0.01*y(t-2)*y(t-2)-0.02*y(t-3)*y(t-2)+0.01*y(t-1)*y(t-3)-0.03*y(t-2)*y(t-3)-0.01*y(t-3)*y(t-3)"
res7_2: io.github.mandar2812.dynaml.graphics.charts.highcharts.Highchart = Highchart(
  List(
    Series(
      List(
        DataPair(0.0, 2.0661407541404424, None, None),
        DataPair(1.0, -0.20279635893745562, None, None),
        DataPair(2.0, 2.0118829440923975, None, None),
        DataPair(3.0, 2.0661407541404424, None, None),
        DataPair(4.0, -0.7668105130011957, None, None),
        DataPair(5.0, -0.5506978934362262, None, None),
        DataPair(6.0, 0.3259708389782872, None, None),
        DataPair(7.0, 0.7550958981227435, None, None),
        DataPair(8.0, 0.24667640048362177, None, None),
        DataPair(9.0, 0.36204346569998375, None, None),
        DataPair(10.0, 

In [9]:
  //Now train a GP-AR model based on the non-linear time series data.
val train_split = 0.4

val markov_chain_train_data = dtfdata
    .dataset(samples_markov.tail.flatMap(_.sliding(p + 1).toSeq))
    .map(h => (DenseVector(h.take(p).toArray), h.last))
    .to_supervised(identityPipe[(DenseVector[Double], Double)])

val markov_chain_first_sample = dtfdata
    .dataset(samples_markov.head.sliding(p + 1).toSeq)
    .map(h => (DenseVector(h.take(p).toArray), h.last))
    .to_supervised(identityPipe[(DenseVector[Double], Double)])
    .partition(train_split)

val markov_chain_samples = dtfdata.tf_dataset(
    markov_chain_train_data.concatenate(
        markov_chain_first_sample.training_dataset
    ),
    markov_chain_first_sample.test_dataset
)

val test_split_size = markov_chain_samples.test_dataset.size

val hyper_prior = Map(
    "bandwidth" -> UniformRV(0.1, 5d),
    "amplitude" -> UniformRV(0.1, 5d),
    "noiseLevel" -> UniformRV(0.1, 5d)
)

gp_prior.hyperPrior_(
     hyper_prior
)

gp_prior.globalOptConfig_(
    Map(
        "gridStep"  -> "0.15",
        "gridSize"  -> "5",
        "globalOpt" -> "GS",
        "policy"    -> "GS",
        "maxIt"     -> "2"
    )
)

val gp_nar_model = gp_prior.posteriorModel(
    markov_chain_samples.training_dataset.data.toSeq
)

val (test_preds, lower_bar, upper_bar) = gp_nar_model
    .predictionWithErrorBars(
        markov_chain_samples.test_dataset
            .map(tup2_1[DenseVector[Double], Double])
            .data
            .toSeq,
        3
    )
    .map(pattern => (pattern._2, pattern._3, pattern._4))
    .unzip3

-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
Grid Search 
-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-

15 / 15 [=======================================================] 100.00 % 1/s  
Optimum value of energy is: 1243.2673431236565
Configuration: 
Map(
  "bandwidth" -> 2.260910792943165,
  "amplitude" -> 0.5413270114642186,
  "noiseLevel" -> 1.836224856766493
)


train_split: Double = 0.4
markov_chain_train_data: io.github.mandar2812.dynaml.tensorflow.data.SupervisedDataSet[DenseVector[Double], Double] = SupervisedDataSet(
  io.github.mandar2812.dynaml.tensorflow.data.DataSet@1a44cb53,
  io.github.mandar2812.dynaml.tensorflow.data.DataSet@23713795
)
markov_chain_first_sample: io.github.mandar2812.dynaml.tensorflow.data.TFDataSet[(DenseVector[Double], Double)] = TFDataSet(
  io.github.mandar2812.dynaml.tensorflow.data.DataSet@31a72273,
  io.github.mandar2812.dynaml.tensorflow.data.DataSet@23c13bf2
)
markov_chain_samples: io.github.mandar2812.dynaml.tensorflow.data.TFDataSet[(DenseVector[Double], Double)] = TFDataSet(
  io.github.mandar2812.dynaml.tensorflow.data.DataSet@49526391,
  io.github.mandar2812.dynaml.tensorflow.data.DataSet@23c13bf2
)
test_split_size: Int = 59
hyper_prior: Map[String, UniformRV] = Map(
  "bandwidth" -> UniformRV(0.1, 5.0),
  "amplitude" -> UniformRV(0.1, 5.0),
  "noiseLevel" -> UniformRV(0.1, 5.0)
)
gp_nar_model: gp_pri

In [10]:
val last_train_sample_index = 
    (train_split * 
         samples_markov.head
            .sliding(p + 1)
            .toSeq
            .length
    ) - 1

spline(xs.zip(samples_markov.head))
hold()
spline(xs.takeRight(test_split_size).zip(test_preds))
spline(xs.takeRight(test_split_size).zip(lower_bar))
spline(xs.takeRight(test_split_size).zip(upper_bar))
unhold()
title("Time Series Prediction")
legend(
    Seq("Time Series", "MAP Prediction", "Lower Error Bar", "Upper Error Bar")
)

Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).
Output written to http://tail-box:42715 (CMD + Click link in Mac OSX).


last_train_sample_index: Double = 37.800000000000004
res9_1: io.github.mandar2812.dynaml.graphics.charts.highcharts.Highchart = Highchart(
  List(
    Series(
      List(
        DataPair(0.0, 2.0661407541404424, None, None),
        DataPair(1.0, -0.20279635893745562, None, None),
        DataPair(2.0, 2.0118829440923975, None, None),
        DataPair(3.0, 2.0661407541404424, None, None),
        DataPair(4.0, -0.7668105130011957, None, None),
        DataPair(5.0, -0.5506978934362262, None, None),
        DataPair(6.0, 0.3259708389782872, None, None),
        DataPair(7.0, 0.7550958981227435, None, None),
        DataPair(8.0, 0.24667640048362177, None, None),
        DataPair(9.0, 0.36204346569998375, None, None),
        DataPair(10.0, 0.5915722688138524, None, None),
        DataPair(11.0, 0.6932176202171723, None, None),
        DataPair(12.0, 0.7861619749847729, None, None),
        DataPair(13.0, 0.6823027547053315, None, None),
        DataPair(14.0, 1.5148898666329573, None, 

In [11]:
stopServer